In [1]:
import numpy as np
import pandas as pd

## AB Test Simulation

* A: CVR 30%
* B: CVR 31%
* C: CVR 32%

In [2]:
A = 0.30
B = 0.31
C = 0.32

products_cvr = [A, B, C]
products_name = ["A", "B", "C"]

## 1. Greedy Algorithm

In [3]:
def greedy(n_iter, products_cvr, products_name, init_popul_per_iter=1000, increment_size=200):
    popul_total = 0
    converted_total = 0
    popul_size_per_iter = [init_popul_per_iter] * len(products_cvr)
    for i in range(n_iter):
        results = []
        for ix, cvr in enumerate(products_cvr):
            popul = popul_size_per_iter[ix]
            converted = np.random.binomial(popul, cvr)

            results.append(converted / popul)
            popul_total += popul
            converted_total += converted
        
        # update population
        max_ix = np.argmax(results)
        min_ix = np.argmin(results)
        popul_size_per_iter[max_ix] = popul_size_per_iter[max_ix] + increment_size
        popul_size_per_iter[min_ix] = popul_size_per_iter[min_ix] - increment_size

        if 0 in popul_size_per_iter:
            zero_ix = popul_size_per_iter.index(0)

            popul_size_per_iter = popul_size_per_iter[:zero_ix] + popul_size_per_iter[zero_ix+1:]
            products_cvr = products_cvr[:zero_ix] + products_cvr[zero_ix+1:]
            products_name = products_name[:zero_ix] + products_name[zero_ix+1:]

        print_str = f"iteration: {i+1:3} | CVR: {100 * converted_total / popul_total:.2f}% | population: "
        for ix, popul_size in enumerate(popul_size_per_iter):
            print_str += f"{products_name[ix]}={popul_size} "
            
        print(print_str)
        

In [4]:
greedy(10, products_cvr, products_name)

iteration:   1 | CVR: 31.67% | population: A=1000 B=1200 C=800 
iteration:   2 | CVR: 30.72% | population: A=800 B=1200 C=1000 
iteration:   3 | CVR: 31.32% | population: A=800 B=1000 C=1200 
iteration:   4 | CVR: 31.62% | population: A=600 B=1000 C=1400 
iteration:   5 | CVR: 31.38% | population: A=400 B=1000 C=1600 
iteration:   6 | CVR: 31.46% | population: A=600 B=800 C=1600 
iteration:   7 | CVR: 31.50% | population: A=600 B=600 C=1800 
iteration:   8 | CVR: 31.59% | population: A=400 B=600 C=2000 
iteration:   9 | CVR: 31.55% | population: A=200 B=600 C=2200 
iteration:  10 | CVR: 31.66% | population: B=600 C=2400 


## 2. Epsilon-Greedy Algorithm

In [5]:
def e_greedy(n_iter, products_cvr, products_name, eps = 0.3, init_popul_per_iter=1000, increment_size=200):
    popul_total = 0
    converted_total = 0
    popul_size_per_iter = [init_popul_per_iter] * len(products_cvr)
    for i in range(n_iter):
        results = []
        for ix, cvr in enumerate(products_cvr):
            popul = popul_size_per_iter[ix]
            converted = np.random.binomial(popul, cvr)

            results.append(converted / popul)
            popul_total += popul
            converted_total += converted
        
        # update population
        if np.random.rand() > eps:
            max_ix = np.argmax(results)
            min_ix = np.argmin(results)
        else:
            max_ix, min_ix = np.random.choice(range(len(popul_size_per_iter)), size=2, replace=False)

        popul_size_per_iter[max_ix] = popul_size_per_iter[max_ix] + increment_size
        popul_size_per_iter[min_ix] = popul_size_per_iter[min_ix] - increment_size

        if 0 in popul_size_per_iter:
            zero_ix = popul_size_per_iter.index(0)

            popul_size_per_iter = popul_size_per_iter[:zero_ix] + popul_size_per_iter[zero_ix+1:]
            products_cvr = products_cvr[:zero_ix] + products_cvr[zero_ix+1:]
            products_name = products_name[:zero_ix] + products_name[zero_ix+1:]

        print_str = f"iteration: {i+1:3} | CVR: {100 * converted_total / popul_total:.2f}% | population: "
        for ix, popul_size in enumerate(popul_size_per_iter):
            print_str += f"{products_name[ix]}={popul_size} "
            
        print(print_str)
        

In [6]:
e_greedy(10, products_cvr, products_name)

iteration:   1 | CVR: 30.07% | population: A=800 B=1200 C=1000 
iteration:   2 | CVR: 30.40% | population: A=800 B=1400 C=800 
iteration:   3 | CVR: 30.36% | population: A=600 B=1400 C=1000 
iteration:   4 | CVR: 30.27% | population: A=400 B=1600 C=1000 
iteration:   5 | CVR: 30.35% | population: A=200 B=1800 C=1000 
iteration:   6 | CVR: 30.66% | population: B=1800 C=1200 
iteration:   7 | CVR: 30.85% | population: B=1600 C=1400 
iteration:   8 | CVR: 30.90% | population: B=1400 C=1600 
iteration:   9 | CVR: 31.01% | population: B=1200 C=1800 
iteration:  10 | CVR: 31.01% | population: B=1000 C=2000 


## 3. UCB (Upper-Confidence-Bound) Algorithm